In [576]:
import numpy as np
import time
import os
import tracemalloc

In [577]:
def screen_clear():
   # for mac and linux(here, os.name is 'posix')
   if os.name == 'posix':
      _ = os.system('clear')
   else:
      # for windows platfrom
      _ = os.system('cls')

In [578]:
def random_input_generator(lower_bound=-100, upper_bound=100, sd=250):
  #Function to generate random inputs to test the counter
  #Python float size is 64 bit. So there are a lot of numbers it can generate between any two limits.
  #Therefore simply calling random numbers between the limits doesn't guarantee repetition in inputs.
  #Hence 50 random numbers were generated. 30 float and 20 integers. This function randomly returns one of the 50 numbers when it's called.
  np.random.seed(seed=250)
  r_array1 = np.random.uniform(lower_bound,upper_bound,30)
  r_array2 = np.random.randint(lower_bound,upper_bound,20)
  r_array = np.concatenate([r_array1,r_array2])
  np.random.seed(seed=None)
  return np.random.choice(r_array)

In [579]:
#buffer and i are the only two global variables.
#buffer keeps track of the last 500 input values.
#i keeps track of how many inputs were provided in total until now.
buffer = np.empty([500])
buffer[:] = np.nan
i = 0

In [580]:
#Lists, tuples and dictionaries are completely avoided to save memory.
#NumPy is written in C, and executes very quickly as a result.
#Hence all storage elements are numpy arrays which are very similar to C arrays.

def count_function(input_value, k=10):
  #The actual counter function that tracks the k most frequent numbers
  global buffer, i

  ###This commented out piece of code is for filling up the buffer for testing purposes
  #while(i < 5):
  #  buffer[:-1] = buffer[1:]; buffer[-1] = random_input_generator()
  #  i = i + 1
  #i = i - 1

  #This if statement is used to add elements to the buffer.
  #First 500 elements fill up the buffer.
  #Afterwards the first element is removed everytime a new element is added.
  if i > 500:
    buffer[:-1] = buffer[1:]; buffer[-1] = input_value
    unique, counts = np.unique(buffer, return_counts=True)
    i = i + 1
  else:
    buffer[:-1] = buffer[1:]; buffer[-1] = input_value
    buffer_new = buffer[~np.isnan(buffer)]
    unique, counts = np.unique(buffer_new, return_counts=True)
    i = i + 1
  
  #Two seperate arrays keep track of the variables (element_array) and its count (count_array)
  #The size of both arrays is k.
  count_array = np.empty([k])
  count_array[:] = np.nan
  element_array = np.empty([k])
  element_array[:] = np.nan
  
  if i < 10:
    k = i
  v = 0
  while(v < k):
    if counts.size == 0:
      #If all the values in the counts container gets deleted, the following np.nanmax function fails.
      #The break statement here avoids that.
      break
    result = np.where(counts == np.nanmax(counts))
    #np.where returns the index value of the largest element in the array.
    #If more largest element repeats multiple times it returns an array of all the corresponding index values.
    for m in result[0]:
      count_array[v] = np.nanmax(counts)
      element_array[v] = unique[m]
      v = v + 1
      if v >= 10:
        break
    counts = np.delete(counts, result[0])
    unique = np.delete(unique, result[0])

  return element_array, count_array

In [ ]:
#Set the number of inputs to be provided.
number_of_inputs = 10000
num = 1
while(num <= 10000):
  #tracemalloc is used to measure memory consumption.
  tracemalloc.start()
  a, b = count_function(random_input_generator())
  current, peak = tracemalloc.get_traced_memory()
  print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
  tracemalloc.stop()
  print("")
  a = a[~np.isnan(a)]
  b = b[~np.isnan(b)]
  for p in range(len(a)):
    print(str(a[p])+": "+str(b[p])+" times")
  print("\n")
  time.sleep(2)
  screen_clear()
  num = num + 1